<a href="https://colab.research.google.com/github/liangli217/LLM_learning/blob/main/Agents_with_OpenAI_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://drive.google.com/uc?export=view&id=1oCldVicFepwdYbPHcD8Ui4VeyQVJYgut" height="400"><center>

In [2]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 15.2 MB/s eta 0:00:00


In [6]:
from openai import OpenAI
from typing import List, Optional, Dict
from dataclasses import dataclass
from agents import Agent, Runner, function_tool

In [5]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
import os
os.environ['OPENAI_API_KEY'] = api_key

#Proper client initialization
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))